In [1]:
import pandas as pd
import pathlib, requests, re
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
import calendar

In [3]:
pathCBHPI = pathlib.Path('../data/raw/others/CBHPI.xlsx')
assert pathCBHPI.exists()

In [5]:
seriesCBHPI = pd.read_excel(pathCBHPI, sheet_name='series', encoding='utf-8', index_col=0)
seriesCBHPI['quarter'] = seriesCBHPI.index.to_period('Q')

In [77]:
seriesReleaseDate = pd.read_excel(pathCBHPI, sheet_name='release dates', encoding='utf-8', index_col=0)
seriesReleaseDate.rename(columns={'New Residential Sales Press Release \n10:00 am': 'vintage'}, inplace=True)
seriesReleaseDate = seriesReleaseDate[seriesReleaseDate.index.month.map(lambda x: x in {3, 6, 9, 12})]
seriesReleaseDate['quarter'] = seriesReleaseDate.index.to_period('Q')

In [94]:
series = pd.merge(seriesCBHPI, seriesReleaseDate[['vintage', 'quarter']], left_on='quarter', right_on='quarter', how='left')
df = pd.DataFrame(index=series['quarter'])

In [95]:
for index, row in series.iterrows():
    if not pd.isnull(row['vintage']):
        col = 'CBHPI_' + row['vintage'].strftime('%Y%m%d')
        values = []
        for value, quarter in zip(series['CBHPI'], series['quarter']):
            if quarter <= row['quarter']:
                values.append(value)
            else:
                values.append(float('nan'))
        df[col] = values

In [100]:
df.index = df.index.map(lambda x: x.start_time.strftime('%m/%d/%Y'))


In [101]:
pathCBHPI2 = pathlib.Path('../data/raw/others/CBHPI2.xlsx')
with pd.ExcelWriter(pathCBHPI2) as writer:
    df.to_excel(writer, sheet_name='dataframe')